1. Загрузите данные load_wine из sklearn.datasets. Из обучающей части исключите объекты класса 2. Обучите случайный лес, задав только гиперпараметры `n_estimators=100` и `random_state=0`. Оцените важность признаков. Укажите название двух наиболее важных признаков. 

In [ ]:
from sklearn.datasets import load_wine
data = load_wine()

In [ ]:
X = data['data'][:130]
y = data['target'][:130]
print(list(y).count(2))
print(len(y))
print(data.feature_names)

0
130
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, test_size=0.4)
clf = RandomForestClassifier(n_estimators = 100, random_state=0)
clf.fit(X_train, y_train)
print(mean_squared_error(y_test, clf.predict(X_test)))
print(mean_squared_error(y_train, clf.predict(X_train)))
clf.feature_importances_

0.019230769230769232
0.0


array([0.21935844, 0.04223309, 0.03581304, 0.02084691, 0.06661592,
       0.02219951, 0.08844076, 0.01349933, 0.01025609, 0.15733857,
       0.00717952, 0.01874221, 0.2974766 ])

proline and alcohol are the most important features

2. Загрузите данные load_wine из sklearn.datasets. Из обучающей части исключите объекты класса 2. Отмасштабируйте признаки, используя класс StandardScaler с гиперпараметрами по умолчанию. Обучите случайный лес, задав только гиперпараметры  `n_estimators = 100` и `random_state=0`. Оцените важность признаков. Укажите название двух наиболее важных признаков.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, test_size=0.4)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)
clf.feature_importances_

array([0.21974422, 0.02078267, 0.01234585, 0.02612943, 0.06872049,
       0.04695951, 0.0975258 , 0.01709101, 0.00341708, 0.16818337,
       0.00838408, 0.02737063, 0.28334586])

The most important features are still proline and alcohol

Ниже приведена неполная реализация класса Bagging который имеет методы `fit` для обучения бэггинга над `DecisionTreeRegressor` и метод `predict` для предсказания. Допишите необходимый код, чтобы реализовать бэггинг.

используемы переменные в коде:
- `self.n_estimators`, `n_estimators` - число используемых деревьев
- `self.regressors` - список объектов класса `DecisionTreeRegressor`, к которым уже был применён метод `fit`
Данный список необъодимо заполнить в методе `fit` и использовать для предсказания в методе `predict`
- `ind`-  выбранные индексы объектов при бутстрапе

при создании объекта класса `DecisionTreeRegressor` зафиксируйте  
`random_state=0`

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
class Bagging():
  def __init__(self, n_estimators=10):
    self.n_estimators = n_estimators
    self.regressors = []
  def fit(self, x_train, y_train):
    for i in range(self.n_estimators):
      np.random.seed(i)
      ind = np.random.choice(np.arange(x_train.shape[0]), size = x_train.shape[0])
      dtr = DecisionTreeRegressor(random_state=0)
      dtr.fit(x_train[ind], y_train[ind])
      self.regressors.append(dtr)
  def predict(self, x_test):
    predictions = np.zeros(x_test.shape[0])
    for i in range(self.n_estimators):
      predictions += self.regressors[i].predict(x_test)
    return predictions    


Загрузите данные приложенные к заданию

In [ ]:
import pandas as pd
from google.colab import files
uploder = files.upload()

Saving data.csv to data.csv


Положим матрицу объекты-признаки в переменную `X`, а ответы в переменную `y`

In [ ]:
Data = pd.read_csv('data.csv', header=None)
X, y = Data.iloc[:, :100], Data.iloc[:, 100]


Положим первые 6000 объектов в обучающую часть, остальные объекты в тестовую часть

In [ ]:
x_train, y_train = X[:6000], y[:6000]
x_test, y_test = X[6000:], y[6000:]

In [ ]:
x_train, y_train, x_test, y_test = x_train.values, y_train.values, x_test.values, y_test.values

In [ ]:
x_train_list = []
for i in range(x_train.shape[0]):
  x_train_list.append(x_train.loc[i].tolist())
x_test_list = []
for i in range(x_test.shape[0]):
  x_test_list.append(x_test.loc[i+6000].tolist())
y_train_list = y_train.tolist()
y_test_list = y_test.tolist()

3. Обучите бэггинг на 1 дереве. Оцените качество по метрике MSE на тестовой части. Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [ ]:
bag = Bagging(n_estimators=1)
bag.fit(x_train, y_train)
mean_squared_error(y_test, bag.predict(x_test)) / 1000

33.010494022313495

4. Обучите бэггинг на 5 деревьях. Оцените качество по метрике MSE на тестовой части. Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [ ]:
bag = Bagging(n_estimators=5)
bag.fit(x_train, y_train)
mean_squared_error(y_test, bag.predict(x_test)/5) / 1000

14.866908232716762

5. Обучите бэггинг на 100 деревьях. Оцените качество по метрике MSE на тестовой части. . Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [ ]:
bag = Bagging(n_estimators=100)
bag.fit(x_train, y_train)
mean_squared_error(y_test, bag.predict(x_test)/100) / 1000

11.010134831667148

6. Обучите на этих же данных случайный лес, используйте гиперпараметр `n_estimators = 1`, зафиксируйте  
`random_state=0`. Оцените качество по метрике MSE на тестовой части. . Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(random_state=0, n_estimators=1)
rfr.fit(x_train, y_train)
mean_squared_error(y_test, rfr.predict(x_test)) / 1000

35.12528641185822

7. Обучите на этих же данных случайный лес, используйте гиперпараметр `n_estimators = 5`, зафиксируйте  
`random_state=0`. Оцените качество по метрике MSE на тестовой части. . Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [ ]:
rfr = RandomForestRegressor(random_state=0, n_estimators=5)
rfr.fit(x_train, y_train)
mean_squared_error(y_test, rfr.predict(x_test)) / 1000

15.367496844538215

8. Обучите на этих же данных случайный лес, используйте гиперпараметр `n_estimators = 100`, зафиксируйте  
`random_state=0`. Оцените качество по метрике MSE на тестовой части. . Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [ ]:
rfr = RandomForestRegressor(random_state=0, n_estimators=100)
rfr.fit(x_train, y_train)
mean_squared_error(y_test, rfr.predict(x_test)) / 1000

10.835490183715155

9. Изучите документацию и разберитесь как посчитать Out-of-bag ошибку в RandomForestRegressor. Обучите RandomForestRegressor с гиперпараметром n_estimators=100 на обучающей части, зафиксируйте  
`random_state=0`. Найдите Out-of-bag ошибку алгоритма. Ответ округлите до сотых.

In [ ]:
rfr = RandomForestRegressor(random_state=0, n_estimators=100, oob_score=True)
rfr.fit(x_train, y_train)
print(rfr.oob_score_)
print(rfr.oob_prediction_)

0.7516660793764367
[  -6.32335677 -155.74796781 -214.95561756 ...  -64.7399867    62.65722668
   31.10586409]
